In [1]:
# %%javascript
# IPython.OutputArea.prototype._should_scroll = function(lines) {
#     return false;
# }

In [2]:
%reload_ext autoreload
%autoreload 1
import pymongo

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from viewing import *
from util import *
%aimport sample
from sample import Sample

In [3]:
# Configuration of data frames
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [4]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client['thesis']
db_data = database['data']

In [5]:
# data_list = list(db_data.find())

# samples_list = [Sample(d) for d in data_list]
boards_list = [b['_id'] for b in db_data.aggregate([{"$group": { "_id": "$Board"} }])]
boards_32 = [b['_id'] for b in db_data.aggregate([{"$match": { "Type": {"$eq": 32} } },
                                                  {"$group": { "_id": "$Board" } }])]
boards_64 = np.setdiff1d(boards_list, boards_32, assume_unique=False).tolist()

# Obtención de las métricas

## Uniformity

`Uniformity` es la relación entre el número de '0' y de '1' que hay en una muestra de datos. El valor ideal está en torno a __50%__.

Matematicamente, se representa por la siguient formula:

$$\text{Uniformity} = \frac{1}{n} \sum\limits_{l=1}^n r_{i,l} \times 100 \%$$

Donde $r_{i, l}$ representa el bit numero _l_ de la muestra y $n$ el número de placas. Nuestras muestras son de 4096 bits y disponemos de 206 placas.

---

Como se puede ver en los resultados, la uniformidad de las muestras está cercana al valor ideal.
Los valores que desvian la media son las primeras muestras de cada placa, esto es, la primera muestra que se toma de la placa una vez se ha programado. Esto es asi para todas las regiones

In [59]:
uniformity_df = pd.DataFrame(columns=['_id', 'Board', 'Type', 'Timestamp', 'Day', 'Month', 'Year', 'H', 'M', 'S', 'Region', 'Uniformity'])

for i, dump in enumerate(db_data.find().limit(12_000)):
    d = sample.Sample(dump)
    uniformity_df = uniformity_df.append({'_id': d._id, 'Board': d.Board, 'Type': d.Type, 'Timestamp': d.Timestamp, 
                                          'Day': d.Datetime.day, 'Month': d.Datetime.month, 'Year': d.Datetime.year,
                                          'H': d.Datetime.hour, 'M': d.Datetime.minute, 'S': d.Datetime.second,
                                          'Region': d.Mem_pos, 'Uniformity': d.uniformity()}, ignore_index=True)
    
    if i % 1000 == 0:
        print(f'Calculated {i} samples')
        
uniformity_df.to_csv('./csv/uniformity.csv', index=False)

Calculated 0 samples
Calculated 1000 samples
Calculated 2000 samples
Calculated 3000 samples
Calculated 4000 samples
Calculated 5000 samples
Calculated 6000 samples
Calculated 7000 samples
Calculated 8000 samples
Calculated 9000 samples
Calculated 10000 samples
Calculated 11000 samples


In [60]:
# import qgrid
from IPython.display import display

# qgrid_widget = qgrid.show_grid(uniformity_df, show_toolbar=True)
# qgrid_widget

uniformity_df.sort_values(['Timestamp', 'Region'], ascending=[True, True], inplace=True)

summary_columns = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
uniformity_summary_df = pd.DataFrame(columns=['Region'] + summary_columns)

uniformity_regions_df = uniformity_df.groupby('Region')
for mem_region, region_group_df in uniformity_regions_df:
    summary_df = region_group_df.describe()
    summary_data = {}
    summary_data['Region'] = mem_region
    for value in summary_columns:
        summary_data[value] = summary_df.loc[value][0]
    uniformity_summary_df = uniformity_summary_df.append(summary_data, ignore_index=True)
    
uniformity_summary_df

,Region,count,mean,std,min,25%,50%,75%,max
0,0x20000000,276.0,5.349687,0.577288,4.418945,4.687500,5.566406,5.639648,6.689453
1,0x20000200,276.0,14.288419,1.822459,10.400391,13.720703,13.964844,13.964844,21.093750
2,0x20000400,275.0,31.256392,3.784717,17.675781,32.080078,32.080078,32.177734,44.213867
3,0x20000600,275.0,39.284712,2.523350,26.611328,38.513184,39.501953,40.209961,44.018555
4,0x20000800,275.0,46.291815,3.774494,33.666992,42.456055,48.193359,49.450684,51.806641
5,0x20000a00,275.0,46.228249,3.478887,41.040039,41.870117,47.827148,49.133301,51.733398
6,0x20000c00,275.0,47.078569,2.778261,41.918945,44.482422,48.388672,49.511719,51.342773
7,0x20000e00,275.0,47.138406,2.477448,43.115234,44.433594,48.071289,49.279785,51.611328
8,0x20001000,274.0,46.244422,3.641193,40.844727,41.650391,48.059082,49.432373,52.001953
9,0x20001200,274.0,46.453813,3.416707,41.552734,42.578125,48.339844,49.389648,51.220703


In [16]:
# qgrid_widget.get_changed_df()

## Bit-aliasing

The design and operation conditions can affect the frequency of the RO responses. As a
240 consequence, some RO responses might give a fixed value (e.g., “00” or “01”) in all devices. To measure
241 this undesirable behaviour the bit-aliasing metric. Mathematically, it is described as follow

## Reliability

## Uniqueness

## 